# Lab 1 GPT code

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

# ----------------------------
# 1. Генерація або завантаження даних
# ----------------------------
def generate_data(n=500, seed=42):
    np.random.seed(seed)
    v = np.random.normal(0, 0.1, n)
    y = np.zeros(n)
    # приклад стабільної ARMA(2,2) системи
    a = [0.5, -0.3]
    b = [0.7, -0.2]
    for t in range(max(len(a), len(b)), n):
        y[t] = (a[0]*y[t-1] + a[1]*y[t-2] +
                b[0]*v[t-1] + b[1]*v[t-2] + v[t])
    return y, v

def load_or_generate():
    y_file, v_file = Path("y_generated.txt"), Path("v_generated.txt")
    if y_file.exists() and v_file.exists():
        y = np.loadtxt(y_file)
        v = np.loadtxt(v_file)
    else:
        y, v = generate_data()
        np.savetxt(y_file, y)
        np.savetxt(v_file, v)
        print("Файли не знайдено. Згенеровано синтетичні дані і збережено.")
    return y, v

y, v = load_or_generate()
print(f"Середнє y: {y.mean():.4f}, σ(y): {y.std():.4f}")

# ----------------------------
# 2. Побудова матриці регресорів
# ----------------------------
def design_matrix(y, v, p, q):
    n = len(y)
    m = max(p, q)
    X, Y = [], []
    for t in range(m, n):
        row = []
        # AR частина
        for i in range(1, p+1):
            row.append(y[t-i])
        # MA частина
        for j in range(1, q+1):
            row.append(v[t-j])
        X.append(row)
        Y.append(y[t])
    return np.array(X), np.array(Y)

# ----------------------------
# 3. Методи оцінки
# ----------------------------
def ols_estimation(X, Y):
    theta = np.linalg.lstsq(X, Y, rcond=None)[0]
    y_hat = X @ theta
    e = Y - y_hat
    return theta, e

def rls_estimation(X, Y, lam=1.0, delta=1000):
    n_samples, n_params = X.shape
    theta = np.zeros(n_params)
    P = delta * np.eye(n_params)
    thetas = np.zeros((n_samples, n_params))
    for k in range(n_samples):
        xk = X[k, :].reshape(-1, 1)
        denom = lam + (xk.T @ P @ xk).item()
        K = (P @ xk / denom).flatten()
        err = (Y[k] - xk.T @ theta.reshape(-1,1)).item()
        theta = theta + K * err
        P = (P - K[:, None] @ xk.T @ P) / lam
        thetas[k] = theta
    e = Y - X @ theta
    return theta, e, thetas

# ----------------------------
# 4. Перебір моделей p=1..3, q=1..3
# ----------------------------
results = []
rls_paths = {}  # для перехідних процесів

for p in range(1,4):
    for q in range(1,4):
        X, Y = design_matrix(y, v, p, q)

        # OLS
        th_ols, e_ols = ols_estimation(X, Y)

        # RLS
        th_rls, e_rls, thetas_path = rls_estimation(X, Y)
        rls_paths[(p,q)] = thetas_path

        n = len(Y)
        k = p + q
        sse_ols = np.sum(e_ols**2)
        sse_rls = np.sum(e_rls**2)
        r2_ols = 1 - sse_ols/np.sum((Y - Y.mean())**2)
        r2_rls = 1 - sse_rls/np.sum((Y - Y.mean())**2)
        aic_ols = n*np.log(sse_ols/n) + 2*k
        aic_rls = n*np.log(sse_rls/n) + 2*k

        results.append({
            'p': p, 'q': q,
            'R2_OLS': r2_ols,
            'R2_RLS': r2_rls,
            'SSE_OLS': sse_ols,
            'SSE_RLS': sse_rls,
            'AIC_OLS': aic_ols,
            'AIC_RLS': aic_rls
        })

res_df = pd.DataFrame(results)
print(res_df)

# ----------------------------
# 5. Графіки перехідного процесу РМНК
# ----------------------------
for (p,q), path in rls_paths.items():
    plt.figure(figsize=(8,4))
    for i in range(path.shape[1]):
        plt.plot(path[:,i], label=f'θ{i+1}')
    plt.title(f'Перехідний процес RLS для моделі ARXS({p},{q})')
    plt.xlabel('Час k')
    plt.ylabel('Оцінка параметра')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# ----------------------------
# 6. Графіки залежностей R2, SSE, AIC
# ----------------------------
def plot_metric(metric_name):
    plt.figure(figsize=(7,4))
    for method in ['OLS','RLS']:
        vals = []
        orders = []
        for _, r in res_df.iterrows():
            orders.append(f"{int(r.p)},{int(r.q)}")
            vals.append(r[f"{metric_name}_{method}"])
        plt.plot(orders, vals, marker='o', label=method)
    plt.title(metric_name + ' vs порядок моделі')
    plt.xlabel('Порядок (p,q)')
    plt.ylabel(metric_name)
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

plot_metric('R2')
plot_metric('SSE')
plot_metric('AIC')

# Збереження у файл
res_df.to_excel('results_models.xlsx', index=False)
print("\nРезультати збережені у results_models.xlsx")
